## House Price Prediction Model


In [ ]:
#importing required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data=pd.read_csv("housing.csv")
data.head()

## Data Exploration

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20433 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20433 non-null  float64
 1   latitude            20433 non-null  float64
 2   housing_median_age  20433 non-null  float64
 3   total_rooms         20433 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20433 non-null  float64
 6   households          20433 non-null  float64
 7   median_income       20433 non-null  float64
 8   median_house_value  20433 non-null  float64
 9   ocean_proximity     20433 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.7+ MB


In [ ]:
data.dropna(subset=['total_bedrooms'], inplace=True)
data.info()

In [ ]:

from sklearn.model_selection import train_test_split

#initialising features and target variable to predict
features=[cols for cols in data.columns]
features.remove("median_house_value")
target=["median_house_value"]

In [ ]:
#spliiting entire dataset into training and testing data

X_train,X_test,y_train,y_test=train_test_split(data[features], data[target], test_size=0.2)

In [ ]:
train_data=X_train.join(y_train)
train_data

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(train_data.corr(), annot=True)

Looking at the heatmap, we can see that median_house_value depends highly on median_income

In [ ]:
train_data.hist(figsize=(12,8))

## Data Preprocessing

Looking at the histograms above, we can see that some features like "total_rooms","total_bedrooms","population","households" are non-Gaussian in nature

In [ ]:
train_data['total_rooms']=np.log(train_data['total_rooms']+1)
train_data['total_bedrooms']=np.log(train_data['total_rooms']+1)
train_data['households']=np.log(train_data['households']+1)
train_data['population']=np.log(train_data['population']+1)

In [ ]:
train_data.hist(figsize=(15,8))

In [ ]:
#Now the ocean_proximity feature has non_numerical values. So, we convert it into binary form.

train_data.ocean_proximity.value_counts()



In [ ]:
train_data=train_data.join(pd.get_dummies(train_data['ocean_proximity'])).drop(['ocean_proximity'],axis=1)

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(train_data.corr(),annot=True)


In [ ]:
train_data['bedroom_ratio']=train_data['total_bedrooms']/train_data['total_rooms']
train_data['rooms_per_household']=train_data['total_rooms']/train_data['households']

In [ ]:
plt.figure(figsize=(14,8))
sns.heatmap(train_data.corr(),annot=True,cmap='YlGnBu')

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X_train,y_train=train_data.drop(["median_house_value"],axis=1),train_data['median_house_value']
model = LinearRegression()
model.fit(X_train,y_train)



LinearRegression()

In [ ]:
X_test=X_test.join(pd.get_dummies(X_test['ocean_proximity'])).drop(['ocean_proximity'],axis=1)
X_test['total_bedrooms'] = np.log(X_test['total_bedrooms'] + 1)
X_test['total_rooms'] = np.log(X_test['total_rooms'] + 1)
X_test['households']=np.log(X_test['households']+1)
X_test['population']=np.log(X_test['population']+1)
X_test['bedroom_ratio']=X_test['total_bedrooms']/X_test['total_rooms']
X_test['rooms_per_household']=X_test['total_rooms']/X_test['households']
test_data=X_test.join(y_test)


In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data
X_test_scaled = scaler.transform(X_test)

# Create a new linear regression model
model_scaled = LinearRegression()

# Train the model on the scaled training data
model_scaled.fit(X_train_scaled, y_train)

# Make predictions on the scaled test data
y_pred_scaled = model_scaled.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred_scaled)
print(f'Mean Squared Error on Test Set: {mse}')


Mean Squared Error on Test Set: 16937589251750.174


In [ ]:
#This error is large and so we should use different regression models to reduce it

Mean Squared Error on Test Set: 16937589251750.174
